In [1]:
from torchvision import models
import torch
dir(models)


['AlexNet',
 'ConvNeXt',
 'DenseNet',
 'EfficientNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'MobileNetV3',
 'RegNet',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 'VisionTransformer',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'convnext',
 'convnext_base',
 'convnext_large',
 'convnext_small',
 'convnext_tiny',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'efficientnet',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'feature_extraction',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3_large',
 '

In [2]:
resnet = models.resnet50(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\grzes/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [3]:
print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [6]:
from PIL import Image
img = Image.open("dog.jpg")

In [10]:
imgt = transform(img)
batch_t = torch.unsqueeze(imgt, 0)


In [12]:
out = resnet(batch_t)
print(out.shape)


torch.Size([1, 1000])


In [14]:
with open('chuj.txt') as f:
  classes = [line.strip() for line in f.readlines()]

In [29]:
_, index = torch.topk(out, 5, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(index)
for i in range(5):
    print(classes[index[0][i]], percentage[index[0][i]].item())

tensor([[208, 209, 207, 273, 163]])
Labrador retriever 52.365989685058594
Chesapeake Bay retriever 7.1881184577941895
golden retriever 5.128643035888672
dingo, warrigal, warragal, Canis dingo 3.665618658065796
bloodhound, sleuthhound 2.7101104259490967


In [33]:
# zamrożenie resnetu
for param in resnet.parameters():
    param.requires_grad=False

In [61]:
[i for i in resnet.layer3[0].conv1.children()]

[]

In [66]:
resnet.layer4[2].conv3

Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [ ]:
class MyNetwork(nn.Module):
    def __init__(self, train, Nclasses, Nlayer=3, Pmax=0.4):
        # call constructor from superclass
        super().__init__()
        
        # define network layers
        self.resnet = models.resnet50(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad=False
        
        self.layers = get_layers(self, train)
        
        self.fcl =
    
    def get_layers(self, train):
        # Musi zwracać coś w tym stylu, tj iterowalny obiekt z N conv layers
        return [resnet.layer4[2].conv3,
                resnet.layer3[2].conv2,
                resnet.layer2[1].conv3]
    def get_featuremaps_idicies(self, train):
        # Musi zwracać obiekt który ma N iterowalnych rzeczy, z których każda ma ileś indeksów feature map. 
        # W sumie ilość indeksów musi być równa ilości klas * Nfeature z 2.3 w artykule
        # Możliwe że się okaże że muszą to być tensory czy coś w tym stylu
        return [[1,35, 63],
               [2,34, 45,48],
               [3, 5, 25, 35]]
        
        
    
    def forward(self, x):
        x 
        return x

# instantiate the model
model = MyNetwork()

# print model architecture
print(model)

In [67]:
from torch.fx import symbolic_trace
# Symbolic tracing frontend - captures the semantics of the module
symbolic_traced : torch.fx.GraphModule = symbolic_trace(resnet)

# High-level intermediate representation (IR) - Graph representation
print(symbolic_traced.graph)


graph():
    %x : torch.Tensor [#users=1] = placeholder[target=x]
    %conv1 : [#users=1] = call_module[target=conv1](args = (%x,), kwargs = {})
    %bn1 : [#users=1] = call_module[target=bn1](args = (%conv1,), kwargs = {})
    %relu : [#users=1] = call_module[target=relu](args = (%bn1,), kwargs = {})
    %maxpool : [#users=2] = call_module[target=maxpool](args = (%relu,), kwargs = {})
    %layer1_0_conv1 : [#users=1] = call_module[target=layer1.0.conv1](args = (%maxpool,), kwargs = {})
    %layer1_0_bn1 : [#users=1] = call_module[target=layer1.0.bn1](args = (%layer1_0_conv1,), kwargs = {})
    %layer1_0_relu : [#users=1] = call_module[target=layer1.0.relu](args = (%layer1_0_bn1,), kwargs = {})
    %layer1_0_conv2 : [#users=1] = call_module[target=layer1.0.conv2](args = (%layer1_0_relu,), kwargs = {})
    %layer1_0_bn2 : [#users=1] = call_module[target=layer1.0.bn2](args = (%layer1_0_conv2,), kwargs = {})
    %layer1_0_relu_1 : [#users=1] = call_module[target=layer1.0.relu](args = (%la

In [68]:
print(symbolic_traced.code)





def forward(self, x : torch.Tensor) -> torch.Tensor:
    conv1 = self.conv1(x);  x = None
    bn1 = self.bn1(conv1);  conv1 = None
    relu = self.relu(bn1);  bn1 = None
    maxpool = self.maxpool(relu);  relu = None
    layer1_0_conv1 = getattr(self.layer1, "0").conv1(maxpool)
    layer1_0_bn1 = getattr(self.layer1, "0").bn1(layer1_0_conv1);  layer1_0_conv1 = None
    layer1_0_relu = getattr(self.layer1, "0").relu(layer1_0_bn1);  layer1_0_bn1 = None
    layer1_0_conv2 = getattr(self.layer1, "0").conv2(layer1_0_relu);  layer1_0_relu = None
    layer1_0_bn2 = getattr(self.layer1, "0").bn2(layer1_0_conv2);  layer1_0_conv2 = None
    layer1_0_relu_1 = getattr(self.layer1, "0").relu(layer1_0_bn2);  layer1_0_bn2 = None
    layer1_0_conv3 = getattr(self.layer1, "0").conv3(layer1_0_relu_1);  layer1_0_relu_1 = None
    layer1_0_bn3 = getattr(self.layer1, "0").bn3(layer1_0_conv3);  layer1_0_conv3 = None
    layer1_0_downsample_0 = getattr(getattr(self.layer1, "0").downsample, "0")(maxpool); 

In [71]:
import torch
import torch.fx

class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.param = torch.nn.Parameter(torch.rand(3, 4))
        self.linear = torch.nn.Linear(4, 5)

    def forward(self, x):
        return torch.topk(torch.sum(
            self.linear(x + self.linear.weight).relu(), dim=-1), 3)

m = MyModule()
gm = torch.fx.symbolic_trace(resnet)

gm.graph.print_tabular()

opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

In [160]:
import torch
import torch.fx as fx
from torch.fx import symbolic_trace
from torch import Tensor
from torch import tensor
from torch import IntTensor
m=models.resnet50(pretrained=True)
def transform(m: torch.nn.Module, n=3, idx=[IntTensor([1,2]),IntTensor([2,3]),IntTensor([3,4])]) -> torch.nn.Module:
    gm : torch.fx.GraphModule = torch.fx.symbolic_trace(m)
    graph=gm.graph
    # FX represents its Graph as an ordered list of
    # nodes, so we can iterate through them.
    IntTensor
    LAYERS = ['layer1_0_conv1', 'layer3_2_conv2', 'layer4_1_conv2']
    FMIDX = [[1,2],[2,3],[3,4]]
    final_nodes=[]
    last_node=None
    
    for node in graph.nodes:
        # do zmiany na tablicę
        if node.name in LAYERS:
            final_nodes.append(node)
        if not last_node and len(final_nodes)==n:
            last_node = node
        if node.name == 'output':
            out_node = node
            
    i=0
    nodes_to_output=[]
    for i in range(n):
        with graph.inserting_after(last_node):
        # Insert a new `call_function` node calling `torch.relu`
            new_node = graph.call_function(torch.tensor,
                                           args=(FMIDX[i],),
                                          kwargs={"dtype":torch.int32})
            last_node = new_node
    
        with graph.inserting_after(last_node):
            new_node = graph.call_function(torch.index_select,
                                          args=(final_nodes[i], 1, last_node))
            last_node = new_node
        
        with graph.inserting_after(last_node):
            new_node = graph.call_function(torch.flatten,
                                          args=(last_node,1))
            nodes_to_output.append(new_node)
            last_node = new_node
    with graph.inserting_after(last_node):
            new_node = graph.call_function(torch.cat,
                                          args=(nodes_to_output,1))
    out_node.args=(new_node,)
    graph.lint() 
    gm.recompile()
    
    return gm
m=transform(m)
m.eval()
m.graph.print_tabular()
sm = symbolic_trace(m)
out = m(batch_t)
print(out.shape)
print(out)
print(sm.code)

opcode         name                   target                                                               args                                    kwargs
-------------  ---------------------  -------------------------------------------------------------------  --------------------------------------  ----------------------
placeholder    x                      x                                                                    ()                                      {}
call_module    conv1                  conv1                                                                (x,)                                    {}
call_module    bn1                    bn1                                                                  (conv1,)                                {}
call_module    relu                   relu                                                                 (bn1,)                                  {}
call_module    maxpool                maxpool                               

torch.Size([1, 6762])
tensor([[-0.4391, -0.2868, -0.2688,  ..., -0.0822, -0.0713, -0.0630]],
       grad_fn=<CatBackward0>)



def forward(self, x : torch.Tensor) -> torch.Tensor:
    conv1 = self.conv1(x);  x = None
    bn1 = self.bn1(conv1);  conv1 = None
    relu = self.relu(bn1);  bn1 = None
    maxpool = self.maxpool(relu);  relu = None
    layer1_0_conv1 = getattr(self.layer1, "0").conv1(maxpool)
    layer1_0_bn1 = getattr(self.layer1, "0").bn1(layer1_0_conv1)
    layer1_0_relu = getattr(self.layer1, "0").relu(layer1_0_bn1);  layer1_0_bn1 = None
    layer1_0_conv2 = getattr(self.layer1, "0").conv2(layer1_0_relu);  layer1_0_relu = None
    layer1_0_bn2 = getattr(self.layer1, "0").bn2(layer1_0_conv2);  layer1_0_conv2 = None
    layer1_0_relu_1 = getattr(self.layer1, "0").relu(layer1_0_bn2);  layer1_0_bn2 = None
    layer1_0_conv3 = getattr(self.layer1, "0").conv3(layer1_0_relu_1);  layer1_0_relu_1 = None
    layer1_0_bn3 = getattr(self.layer1, "0").bn3(layer1_0_conv3);  layer1_0_co

tensor([1., 2.])
